# Anonymizing Data Project


In [68]:
import pandas as pd
import numpy as np
import scipy.stats
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder
# get rid of warnings
import warnings
warnings.filterwarnings("ignore")
# get more than one output per Jupyter cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# for functions we implement later
#from utils import plot_result
import scipy.stats as st
from scipy.stats import ncf




### Function for Perturbation

In [2]:
# function from https://github.com/r0f1/dev_to_posts/blob/master/fake_data/utils.py 
#Florian Rohrer


def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for idx, distribution in enumerate(DISTRIBUTIONS):

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass
        
    return (best_distribution.name, best_params)

In [101]:
# Function to generate pertebation data based on the above function

def generate_like_df(df, categorical_cols, continuous_cols, best_distributions_name, best_distribution_parm, n, seed=0):
    np.random.seed(seed)
    d = {}

    for c in categorical_cols:
        counts = df[c].value_counts()
        d[c] = np.random.choice(list(counts.index), p=(counts/len(df)).values, size=n)
    cn=0
    for c, bdn, bdp in zip(continuous_cols, best_distributions_name,best_distribution_parm):
        cn=cn+1
        print(cn)
        dist =  best_distributions_name
        print(dist)
        print (best_distribution_parm)
        d[c] = make_pdf(ncf,best_fit_params,n)



    return pd.DataFrame(d, columns=categorical_cols+continuous_cols)

In [100]:
def make_pdf(dist, params, size=10000):
    """Generate distributions's Propbability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [47]:


[x[0] for x in best_distributions]








['ncf']

In [32]:
"".join(best_distributions)

TypeError: sequence item 0: expected str instance, tuple found

### Anonymizing Data Process 

In [5]:

#import file into DF
Employee_data_DF = pd.read_csv('https://raw.githubusercontent.com/skhayden/MSDS-7349-Sec403-/master/Class_Project/Employee_data.csv',low_memory=False)
#Remove unneeded columns
Employee_data_DF.drop(['Job.1','Unnamed: 0'], axis = 1, inplace = True) 
Employee_data_DF.head()

First Name Last Name Date of Birth                                     Job  \
0    Matthew   Johnson     4/11/1993  Research scientist (physical sciences)   
1  Elizabeth   Frazier     6/17/1989          Therapist, speech and language   
2       Lori    Vargas     8/24/1959                     Designer, furniture   
3      Sarah  Espinoza     12/1/1983                          Water engineer   
4    Melissa    Hunter     9/24/1949                   Agricultural engineer   

                                             Address  Salary          SSN  \
0                           USS Walker\nFPO AA 91930   64822  691-17-3378   
1  8283 Kimberly Streets Apt. 001\nEast Edward, D...   37351  675-40-3501   
2             235 Devin Ridges\nCraneshire, OR 16065   74379  821-81-4954   
3           78762 Eileen Camp\nNelsonhaven, ID 36069   81275  124-10-4970   
4           3346 Anderson Lakes\nShawville, IL 85803   35409  789-99-6633   

  Employee_Id   Office_Location  Building_number         Phone_Number  \
0      P37429       West Andrew              961           3298685475   
1      P90953  South Tammymouth              590    524-601-7567x7090   
2      P38017       West Arthur              198   316-095-9037x69942   
3      P91959      Greerborough            10033  +1-209-595-5257x826   
4      P36920        Davisburgh              633             44421409   

                        Email  
0            Matthewgmail.com  
1          EFrazier93erez.com  
2      Loriegregory@smith.com  
3        SEspinozad-perry.com  
4  Melissazkimberly@wiley.com

In [6]:
# defines two types of columns; categorical and continuous 
#https://dev.to/r0f1/a-simple-way-to-anonymize-data-with-python-and-pandas-79g
#del categorical_cols
categorical_cols = []

#del Perterbation_col
Perterbation_col = []

#Perterbation_col.append(Employee_data_DF['Salary'])   


for c in list(Employee_data_DF):
    #col = df[c]
    if c != 'Salary':
        categorical_cols.append(c)
        
    else:
        Perterbation_col.append(c)
        






##### Suppresion and Bucketization columns

In [7]:
# suppress columns 
Suppresrion_col = ['Last Name','SSN','Employee_Id']
for c in categorical_cols:
    if c in Suppresrion_col:
        Employee_data_DF[c] = '#####'
        
Employee_data_DF.head()
    

First Name Last Name Date of Birth                                     Job  \
0    Matthew     #####     4/11/1993  Research scientist (physical sciences)   
1  Elizabeth     #####     6/17/1989          Therapist, speech and language   
2       Lori     #####     8/24/1959                     Designer, furniture   
3      Sarah     #####     12/1/1983                          Water engineer   
4    Melissa     #####     9/24/1949                   Agricultural engineer   

                                             Address  Salary    SSN  \
0                           USS Walker\nFPO AA 91930   64822  #####   
1  8283 Kimberly Streets Apt. 001\nEast Edward, D...   37351  #####   
2             235 Devin Ridges\nCraneshire, OR 16065   74379  #####   
3           78762 Eileen Camp\nNelsonhaven, ID 36069   81275  #####   
4           3346 Anderson Lakes\nShawville, IL 85803   35409  #####   

  Employee_Id   Office_Location  Building_number         Phone_Number  \
0       #####       West Andrew              961           3298685475   
1       #####  South Tammymouth              590    524-601-7567x7090   
2       #####       West Arthur              198   316-095-9037x69942   
3       #####      Greerborough            10033  +1-209-595-5257x826   
4       #####        Davisburgh              633             44421409   

                        Email  
0            Matthewgmail.com  
1          EFrazier93erez.com  
2      Loriegregory@smith.com  
3        SEspinozad-perry.com  
4  Melissazkimberly@wiley.com

##### Perturbation columns

In [8]:
#SKIP
best_distributions = []
for c in Perterbation_col:
    data = Employee_data_DF[c]
    best_fit_name, best_fit_params = best_fit_distribution(data, 50)
    best_distributions.append((best_fit_name, best_fit_params))

In [95]:
#data = Employee_data_DF[Perterbation_col]
best_fit_name

'ncf'

In [80]:

    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for idx, distribution in enumerate(DISTRIBUTIONS):

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

SyntaxError: unexpected EOF while parsing (<ipython-input-80-eae1aeb5e6a4>, line 45)

ValueError: dfnum <= 1

In [88]:
e=make_pdf(ncf,best_fit_params)


In [91]:
e.head()

29682.082319    0.000003
29692.627161    0.000003
29703.172002    0.000003
29713.716844    0.000003
29724.261686    0.000003
dtype: float64

In [104]:
gendf = generate_like_df(Employee_data_DF, categorical_cols, Perterbation_col, best_fit_name ,best_fit_params, n= len(Employee_data_DF))
gendf.shape
gendf.head()


1
ncf
(0.11409665500844324, 30.849957076289563, 110.91872250550868, -0.83899714834432082, 63.05088674592389)


(20591, 12)

First Name Last Name Date of Birth  \
29682.082319        Brenda     #####     4/22/1961   
29687.203148       Michele     #####      6/2/1944   
29692.323977         Maria     #####     3/21/1976   
29697.444807     Katherine     #####      5/7/1998   
29702.565636       Kenneth     #####     7/21/1969   
29707.686465        Travis     #####     2/16/1964   
29712.807294        Andrea     #####     11/4/1944   
29717.928123         Allen     #####     6/14/1984   
29723.048952         Wyatt     #####    12/19/1983   
29728.169781        Nicole     #####     4/22/1955   
29733.290610         Tanya     #####     5/27/1966   
29738.411439       Crystal     #####     12/8/1959   
29743.532268          Lori     #####      8/3/1986   
29748.653097       Darrell     #####     5/11/1982   
29753.773927          John     #####     8/25/1990   
29758.894756   Christopher     #####    10/29/1974   
29764.015585       Michael     #####      6/2/1997   
29769.136414         Blake     #####      6/5/1960   
29774.257243      Mitchell     #####    11/29/1972   
29779.378072         Terri     #####     6/20/1994   
29784.498901        Ariana     #####    12/27/1990   
29789.619730         Billy     #####      8/7/1950   
29794.740559      Brittany     #####    10/23/1964   
29799.861388        Philip     #####     10/4/1947   
29804.982217       Matthew     #####     8/10/1964   
29810.103046         Nancy     #####    11/29/1985   
29815.223876       Jessica     #####      9/5/1988   
29820.344705       Bethany     #####      4/7/1951   
29825.465534       Gregory     #####     9/28/1979   
29830.586363         Susan     #####     5/18/1966   
...                    ...       ...           ...   
134971.449180         Mary     #####     7/11/1990   
134976.570009       Brooke     #####     9/27/1975   
134981.690838         Tina     #####    11/27/1958   
134986.811667     Jonathan     #####     9/20/1964   
134991.932496       Joseph     #####     9/21/1976   
134997.053325        Erica     #####     8/11/1996   
135002.174154         Tina     #####     9/24/1983   
135007.294983     Jennifer     #####     7/22/1952   
135012.415812      Anthony     #####     12/8/1975   
135017.536641      Jeffrey     #####    11/22/1973   
135022.657471        Linda     #####    10/11/1962   
135027.778300      Rebekah     #####      7/4/1976   
135032.899129       Alicia     #####     6/21/1985   
135038.019958      Michael     #####     9/18/1992   
135043.140787       Alison     #####     7/11/1987   
135048.261616        David     #####      3/5/1975   
135053.382445        Jesus     #####    10/13/1981   
135058.503274      Kenneth     #####     7/29/1956   
135063.624103        Andre     #####      1/5/1999   
135068.744932   Jacqueline     #####     9/10/1965   
135073.865761      Brandon     #####     7/26/1989   
135078.986591        Tracy     #####      2/5/1951   
135084.107420  Christopher     #####     7/12/1988   
135089.228249      Michael     #####     10/2/1988   
135094.349078        Brian     #####      7/7/1985   
135099.469907        Ethan     #####      1/8/1969   
135104.590736        Jacob     #####     1/30/1970   
135109.711565         Tony     #####    11/24/1967   
135114.832394         Ryan     #####     10/8/1998   
135119.953223         Dana     #####     4/19/1991   

                                                             Job  \
29682.082319                               Travel agency manager   
29687.203148                              Manufacturing engineer   
29692.323977       Chartered legal executive (England and Wales)   
29697.444807                                          Astronomer   
29702.565636                                         Adult nurse   
29707.686465                 Exhibitions officer, museum/gallery   
29712.807294                                    Paediatric nurse   
29717.928123                                 Magazine journalist   
29723.048952                              Applications 

In [ ]:
len(Employee_data_DF)

In [107]:
Employee_data_DF

First Name Last Name Date of Birth  \
0          Matthew     #####     4/11/1993   
1        Elizabeth     #####     6/17/1989   
2             Lori     #####     8/24/1959   
3            Sarah     #####     12/1/1983   
4          Melissa     #####     9/24/1949   
5           Ronnie     #####     2/16/1983   
6           Teresa     #####     2/20/1973   
7        Christina     #####     4/27/1974   
8           Robert     #####      7/8/1954   
9          William     #####      6/5/1946   
10           Sarah     #####      5/5/1987   
11            Mary     #####     9/14/1962   
12           Bryan     #####     3/12/1949   
13          Stacey     #####     5/27/1982   
14           David     #####     5/10/1972   
15          Amanda     #####      6/2/1948   
16         Cassidy     #####    11/23/1994   
17         Yolanda     #####    11/13/1991   
18         Patrick     #####     2/12/1955   
19         Melissa     #####     7/26/1992   
20         Colleen     #####     1/29/1984   
21            Eric     #####     5/17/1988   
22           James     #####    10/30/1994   
23          Nathan     #####     7/25/1966   
24          Pamela     #####    12/10/1962   
25       Katherine     #####     5/22/1996   
26          Robert     #####     6/15/1984   
27          Monica     #####     4/21/1997   
28           Chloe     #####      6/1/1962   
29     Christopher     #####     4/11/1946   
...            ...       ...           ...   
20561        Aaron     #####     1/14/1961   
20562       Andrew     #####     9/17/1968   
20563    Catherine     #####      4/2/1977   
20564      Adriana     #####    11/10/1979   
20565      Kristin     #####      9/5/1984   
20566        David     #####     1/27/1980   
20567       Thomas     #####    11/14/1987   
20568      Jeffrey     #####     12/7/1994   
20569      Anthony     #####     7/15/1947   
20570         Carl     #####      6/3/1982   
20571  Christopher     #####    12/20/1995   
20572         Mary     #####     3/14/1970   
20573    Christina     #####     8/22/1982   
20574        Brian     #####     10/6/1966   
20575         Erin     #####     7/30/1944   
20576       Sherry     #####    10/15/1976   
20577       Sandra     #####      7/4/1959   
20578    Elizabeth     #####      8/7/1965   
20579         Chad     #####      2/3/1979   
20580        Keith     #####     2/17/1988   
20581       Jeremy     #####      3/3/1965   
20582     Nicholas     #####     5/11/1978   
20583       Victor     #####     6/10/1980   
20584       Norman     #####     4/15/1992   
20585         Sara     #####      1/6/1997   
20586    Stephanie     #####     12/4/1989   
20587        Brian     #####      2/6/1991   
20588        Caleb     #####     4/23/1964   
20589       Taylor     #####      4/4/1974   
20590       Robert     #####     1/18/1989   

                                                     Job  \
0                 Research scientist (physical sciences)   
1                         Therapist, speech and language   
2                                    Designer, furniture   
3                                         Water engineer   
4                                  Agricultural engineer   
5                                   Merchandiser, retail   
6                                      Company secretary   
7                                     Restaurant manager   
8                                Forest/woodland manager   
9                                  Child psychotherapist   
10                      Higher education careers adviser   
11                            Further education lecturer   
12                                Occupational hygienist   
13                                   Designer, furniture   
14                                       Event organiser   
15                               Copywriter, advertising   
16                                    Personal assistant   
17                           Lecturer, further education   
18             